# Part 6: Wiggle, Wiggle
OK, so our network is not getting to a low loss, or at least not very quickly, as we add new colors. It's kind of worrisome really... how will it perform with 48 or 96 or the full 120 crayon box of colors?

Let's try something before we add new colors. There's a trick in neural network training that lets you take your existing data and create a lot more of it by doing a bit of coding work and letting the computer create more "free" data for you. Neural networks like lots of examples - you'll spend more time training the network, but much of the time you can get a better training result afterward.

In the case of our data, our color names and RGB values, we can "wiggle" the R, G, and B numbers a bit and still have the same color. Think about it: If gray is (0.5, 0.5, 0.5), your eye can't see much if any difference vs. (0.49, 0.49, 0.49) or (0.51, 0.51, 0.51) or (0.48, 0.48, 0.48). We could easily wiggle each of our RGB triples and call it the same color. Also note we can wiggle each R or G or B independently, so we could get (0.48, 0.50, 0.51) and still call it gray.

Let's see what happens. First, we copy our previous network code, with a few tweaks. Challenge: see if you can spot what I changed vs. the last lesson; there are 2 changes to find.

In [1]:
from keras.layers import Activation, Dense, Dropout
from keras.models import Sequential
import keras.optimizers, keras.utils, numpy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

def train(rgbValues, colorNames, epochs = 16):
    """
    Trains a neural network to understand how to map color names to RGB triples.
    The provided lists of RGB triples must be floating point triples with each
    value in the range [0.0, 1.0], and the number of color names must be the same length.
    Different names are allowed to map to the same RGB triple.
    Returns a trained model that can be used for recognize().
    """

    # Convert the Python map RGB values into a numpy array needed for training.
    rgbNumpyArray = numpy.array(rgbValues, numpy.float64)
    
    # Convert the color labels into a one-hot feature array.
    # Text labels for each array position are in the classes_ list on the binarizer.
    labelBinarizer = LabelBinarizer()
    oneHotLabels = labelBinarizer.fit_transform(colorNames)
    numColors = len(labelBinarizer.classes_)
    colorLabels = labelBinarizer.classes_
    
    # Hyperparameters to define the network shape.
    numFullyConnectedPerceptrons = numColors * 8
    batchSize = 1
    
    model = Sequential([
        # Layer 1: Fully connected layer with ReLU activation.
        Dense(numFullyConnectedPerceptrons, activation='relu', kernel_initializer='TruncatedNormal', input_shape=(3,)),

        # Outputs: SoftMax activation to get probabilities by color.
        Dense(numColors, activation='softmax')
    ])

    print(model.summary())

    # Compile for categorization.
    model.compile(
        optimizer = keras.optimizers.SGD(lr = 0.01, momentum = 0.9, decay = 1e-6, nesterov = True),
        loss = 'categorical_crossentropy',
        metrics = [ 'accuracy' ])

    history = model.fit(rgbNumpyArray, oneHotLabels, epochs=epochs, batch_size=batchSize)

    return (model, colorLabels)

Using TensorFlow backend.


Next, let's add some "wiggle" code. Wiggling is also called *perturbing* the data.

In [2]:
def createMoreTrainingData(colorNameToRGBMap):
    # The incoming color map is not typically going to be oversubscribed with e.g.
    # extra 'red' samples pointing to slightly different colors. We generate a
    # training dataset by perturbing each color by a small amount positive and
    # negative. We do this for each color individually, by pairs, and for all three
    # at once, for each positive and negative value, resulting in dataset that is
    # many times as large.
    perturbValues = [ 0.0, 0.01, 0.02, 0.03 ]
    rgbValues = []
    labels = []
    for colorName, rgb in colorNameToRGBMap.items():
        reds = []
        greens = []
        blues = []
        for perturb in perturbValues:
            if rgb[0] + perturb <= 1.0:
                reds.append(rgb[0] + perturb)
            if perturb != 0.0 and rgb[0] - perturb >= 0.0:
                reds.append(rgb[0] - perturb)
            if rgb[1] + perturb <= 1.0:
                greens.append(rgb[1] + perturb)
            if perturb != 0.0 and rgb[1] - perturb >= 0.0:
                greens.append(rgb[1] - perturb)
            if rgb[2] + perturb <= 1.0:
                blues.append(rgb[2] + perturb)
            if perturb != 0.0 and rgb[2] - perturb >= 0.0:
                blues.append(rgb[2] - perturb)
        for red in reds:
            for green in greens:
                for blue in blues:
                    rgbValues.append((red, green, blue))
                    labels.append(colorName)
    return (rgbValues, labels)

What this function does is it splits the color name to RGB map into the color names and the RGB values, then proceeds to wiggle each of the R, G, and B values up to 0.03 difference from the original. What we get out is a much larger pair of lists, the RGB values with all the wiggles included, and the corresponding names, which have a lot of repeats since we made a lot of different values for gray, black, apricot, and so on.

Let's feed the function a white value (0.97, 0.97, 0.97) and see how many new white values we get back:

In [9]:
(rgbValues, labels) = createMoreTrainingData({ 'white': (0.97, 0.97, 0.97)})
print("\n".join([ str(c) for c in rgbValues ]))

(0.97, 0.97, 0.97)
(0.97, 0.97, 0.98)
(0.97, 0.97, 0.96)
(0.97, 0.97, 0.99)
(0.97, 0.97, 0.95)
(0.97, 0.97, 1.0)
(0.97, 0.97, 0.94)
(0.97, 0.98, 0.97)
(0.97, 0.98, 0.98)
(0.97, 0.98, 0.96)
(0.97, 0.98, 0.99)
(0.97, 0.98, 0.95)
(0.97, 0.98, 1.0)
(0.97, 0.98, 0.94)
(0.97, 0.96, 0.97)
(0.97, 0.96, 0.98)
(0.97, 0.96, 0.96)
(0.97, 0.96, 0.99)
(0.97, 0.96, 0.95)
(0.97, 0.96, 1.0)
(0.97, 0.96, 0.94)
(0.97, 0.99, 0.97)
(0.97, 0.99, 0.98)
(0.97, 0.99, 0.96)
(0.97, 0.99, 0.99)
(0.97, 0.99, 0.95)
(0.97, 0.99, 1.0)
(0.97, 0.99, 0.94)
(0.97, 0.95, 0.97)
(0.97, 0.95, 0.98)
(0.97, 0.95, 0.96)
(0.97, 0.95, 0.99)
(0.97, 0.95, 0.95)
(0.97, 0.95, 1.0)
(0.97, 0.95, 0.94)
(0.97, 1.0, 0.97)
(0.97, 1.0, 0.98)
(0.97, 1.0, 0.96)
(0.97, 1.0, 0.99)
(0.97, 1.0, 0.95)
(0.97, 1.0, 1.0)
(0.97, 1.0, 0.94)
(0.97, 0.94, 0.97)
(0.97, 0.94, 0.98)
(0.97, 0.94, 0.96)
(0.97, 0.94, 0.99)
(0.97, 0.94, 0.95)
(0.97, 0.94, 1.0)
(0.97, 0.94, 0.94)
(0.98, 0.97, 0.97)
(0.98, 0.97, 0.98)
(0.98, 0.97, 0.96)
(0.98, 0.97, 0.99)
(0.98, 

That's just the result for one color, and we're about to give it a lot more. That's a lot of new sample data!

OK, let's put the wiggle code to work on our 24-color crayon box. **Important**: Note for this first sample training I set the number of epochs pretty low, 3. You'll see why...

In [10]:
def rgbToFloat(r, g, b):  # r, g, b in 0-255 range
    return (float(r) / 255.0, float(g) / 255.0, float(b) / 255.0)

colorMap = {
    # 8-crayon box colors
    'red': rgbToFloat(238, 32, 77),
    'yellow': rgbToFloat(252, 232, 131),
    'blue': rgbToFloat(31, 117, 254),
    'brown': rgbToFloat(180, 103, 77),
    'orange': rgbToFloat(255, 117, 56),
    'green': rgbToFloat(28, 172, 20),
    'violet': rgbToFloat(146, 110, 174),
    'black': rgbToFloat(35, 35, 35),

    # Additional for 16-count box
    'red-violet': rgbToFloat(192, 68, 143),
    'red-orange': rgbToFloat(255, 117, 56),
    'yellow-green': rgbToFloat(197, 227, 132),
    'blue-violet': rgbToFloat(115, 102, 189),
    'carnation-pink': rgbToFloat(255, 170, 204),
    'yellow-orange': rgbToFloat(255, 182, 83),
    'blue-green': rgbToFloat(25, 158, 189),
    'white': rgbToFloat(237, 237, 237),

    # Additional for 24-count box
    'violet-red': rgbToFloat(247, 83 ,148),
    'apricot': rgbToFloat(253, 217, 181),
    'cerulean': rgbToFloat(29, 172, 214),
    'indigo': rgbToFloat(93, 118, 203),
    'scarlet': rgbToFloat(242, 40, 71),
    'green-yellow': rgbToFloat(240, 232, 145),
    'bluetiful': rgbToFloat(46, 80, 144),
    'gray': rgbToFloat(149, 145, 140),
}

(rgbValues, colorNames) = createMoreTrainingData(colorMap)
(colorModel, colorLabels) = train(rgbValues, colorNames, 3)

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 192)               768       
_________________________________________________________________
dense_2 (Dense)              (None, 24)                4632      
Total params: 5,400
Trainable params: 5,400
Non-trainable params: 0
_________________________________________________________________
None
Instructions for updating:
Use tf.cast instead.
Epoch 1/3
7252/7252 [==============================] - 8s 1ms/step - loss: 0.8884 - acc: 0.7129
Epoch 2/3
7252/7252 [==============================] - 6s 838us/step - loss: 0.3540 - acc: 0.8729
Epoch 3/3
7252/7252 [==============================] - 6s 826us/step - loss: 0.2881 - acc: 0.8895


OK we only trained 3 epochs and they took a long time, but note even the first epoch got us down to a loss of 0.9, and the second dropped a lot more to about 0.35. By the third we are getting closer to 0.25. Pretty good learning! Let's see how it performs by testing with the sliders:

In [11]:
from ipywidgets import interact
from IPython.core.display import display, HTML

def displayColor(r, g, b):
    rInt = min(255, max(0, int(r * 255.0)))
    gInt = min(255, max(0, int(g * 255.0)))
    bInt = min(255, max(0, int(b * 255.0)))
    hexColor = "#%02X%02X%02X" % (rInt, gInt, bInt)
    display(HTML('<div style="width: 50%; height: 50px; background: ' + hexColor + ';"></div>'))

numPredictionsToShow = 5
@interact(r = (0.0, 1.0, 0.01), g = (0.0, 1.0, 0.01), b = (0.0, 1.0, 0.01))
def getTopPredictionsFromModel(r, g, b):
    testColor = numpy.array([ (r, g, b) ])
    predictions = colorModel.predict(testColor, verbose=0)  # Predictions shape (1, numColors)
    predictions *= 100.0
    predColorTuples = []
    for i in range(0, len(colorLabels)):
        predColorTuples.append((predictions[0][i], colorLabels[i]))
    predAndNames = numpy.array(predColorTuples, dtype=[('pred', float), ('colorName', 'U50')])
    sorted = numpy.sort(predAndNames, order=['pred', 'colorName'])
    sorted = sorted[::-1]  # reverse rows to get highest on top
    for i in range(0, numPredictionsToShow):
        print("%2.1f" % sorted[i][0] + "%", sorted[i][1])
    displayColor(r, g, b)


interactive(children=(FloatSlider(value=0.5, description='r', max=1.0, step=0.01), FloatSlider(value=0.5, desc…

OK, the usual slider is below to let you play around with the number of epochs. I set the default to 5 with a max 20 so you don't wait all day. Waiting all day is why neural network people buy expensive, specialized hardware to speed up the math used for all this. If a few colors are causing this much waiting, just think of what training a million pictures each containing a million pixels must be like...

In [12]:
@interact(epochs = (1, 10))
def trainModel(epochs=5):
    global colorModel
    global colorLabels
    (colorModel, colorLabels) = train(rgbValues, colorNames, epochs=epochs)

interactive(children=(IntSlider(value=5, description='epochs', max=10, min=1), Output()), _dom_classes=('widge…

In [13]:
interact(getTopPredictionsFromModel, r = (0.0, 1.0, 0.01), g = (0.0, 1.0, 0.01), b = (0.0, 1.0, 0.01))

interactive(children=(FloatSlider(value=0.5, description='r', max=1.0, step=0.01), FloatSlider(value=0.5, desc…

<function __main__.getTopPredictionsFromModel(r, g, b)>

OK, so we generated a lot of very similar colors by wiggling or perturbing the original colors, and gave them the same names as the originals, and though training took awhile we get pretty good results after 3 epochs, and even better if we go for 10 or 15 or 20. And we have a new tool in our learning toolbox.

### Coming up...
Time for more... More... MOAR!! crayon colors, with perturbing. Let's see how much slower things get, while seeing if we can get closer to our ideal of having a network be able to examine a color and give it one or more name guesses that are pretty accurate.